In [1]:
from googlesearch import search 

In [2]:
query = "amazon.it/8871527747"

In [3]:
for j in search(query, tld="co.in", num=1, stop=1, pause=2):
    print(j)

https://www.amazon.it/povero-Quello-ricchi-insegnano-denaro/dp/8871527747
